<a href="https://colab.research.google.com/github/Rogerio-mack/IMT_CD_2024/blob/main/IMT_Lab_Estimadores_Logistic_Knn_solucao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->


# **Lab: Hot Encode, Normalização, Regressão Logística e K-Vizinhos mais Próximos**

## Outras questões de revisão:

1. Aqui, nos exercícios, o hot encode e normalização para aplicação dos modelos já está pronto ou não é pedida. Entenda que você deve ser capaz de desenvolver o pipeline completo.
2. Entenda que os resultados de um único conjunto de teste não são suficientes para avaliar os modelos.
3. Sabe como extrair os coeficientes do modelo de regressão logística?
4. Sabe como aplicar a normalização dos dados sem *vazamento* de informações para o conjunto de teste? Aliás, sabe o que é esse *vazamento*?
5. Além da acuracidade podemos empregar outra métrica para avaliar entre dois modelos que competem? Isso faz sentido?
6. Posso empregar outras funções distância no modelo de K-Vizinhos mais Próximos?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
#@markdown Prepare Datasets - **Just Run Before Start**
df = sns.load_dataset('tips')
from sklearn.model_selection import train_test_split

df, df_test = train_test_split(df, test_size=3, random_state=42)

df = df.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

df.to_csv('tips_train.csv', index=False)
df_test.to_csv('tips_test.csv', index=False)

df = sns.load_dataset('tips')
from sklearn.model_selection import train_test_split

df['tip_label'] = np.where(df['tip'] / df['total_bill'] > 0.12, '> 0.12', '<=0.12')
df['credit_card'] = np.random.randint(0, 10000, df.shape[0])
df['Sunday'] = df['day'] == 'Sun'
df['Sunday'] = df['Sunday'].astype(int)

df.drop(['tip', 'smoker', 'day'], axis=1, inplace=True)
df = df[['credit_card', 'total_bill', 'Sunday', 'sex', 'time', 'size', 'tip_label']]

df, df_test = train_test_split(df, test_size=3, random_state=42)
df = df.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

df['sex'] = (df['sex'] == 'Female').astype('int')
df['time'] = (df['time'] == 'Dinner').astype('int')
df_test['sex'] = (df_test['sex'] == 'Female').astype('int')
df_test['time'] = (df_test['time'] == 'Dinner').astype('int')

df.to_csv('tips_train_2.csv', index=False)
df_test.to_csv('tips_test_2.csv', index=False)

In [ ]:
df = pd.read_csv('tips_train.csv')
df_test = pd.read_csv('tips_test.csv')
display(df.head())
display(df_test.head())

,total_bill,tip,sex,smoker,day,time,size
0,25.89,5.16,Male,Yes,Sat,Dinner,4
1,13.00,2.00,Female,Yes,Thur,Lunch,2
2,17.89,2.00,Male,Yes,Sun,Dinner,2
3,28.44,2.56,Male,Yes,Thur,Lunch,2
4,12.48,2.52,Female,No,Thur,Lunch,2


,total_bill,tip,sex,smoker,day,time,size
0,19.82,3.18,Male,No,Sat,Dinner,2
1,8.77,2.00,Male,No,Sun,Dinner,2
2,24.55,2.00,Male,No,Sun,Dinner,4


# 1. Encode

Transforme `df` (`tips_train.csv`) aplicando o hot encode dos atributos não numéricos. Com o mesmo estimador gerado, faça a mesma transformação em `df_test` (`tips_test.csv`).

**Q1.** Qual o número final de atributos em `df`?

**Q2.** Qual a soma de todos os valores de `df_test` transformado? (`df_test.sum().sum()`).


In [ ]:
from sklearn.preprocessing import OneHotEncoder

categorical_cols = df.select_dtypes(include=['category', 'object']).columns

# define
encoder = OneHotEncoder(handle_unknown='ignore', drop='first', sparse_output=False)
encoder.fit(df[categorical_cols])

# transform
encoded_data = encoder.transform(df[categorical_cols])

# dataframe
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))
df_encoded = pd.concat([df, encoded_df], axis=1)
df_encoded = df_encoded.drop(categorical_cols, axis=1)

df_encoded.head()

,total_bill,tip,size,sex_Male,smoker_Yes,day_Sat,day_Sun,day_Thur,time_Lunch
0,25.89,5.16,4,1.0,1.0,1.0,0.0,0.0,0.0
1,13.00,2.00,2,0.0,1.0,0.0,0.0,1.0,1.0
2,17.89,2.00,2,1.0,1.0,0.0,1.0,0.0,0.0
3,28.44,2.56,2,1.0,1.0,0.0,0.0,1.0,1.0
4,12.48,2.52,2,0.0,0.0,0.0,0.0,1.0,1.0


In [ ]:
len(df_encoded.columns)

9

In [ ]:
encoded_data = encoder.transform(df_test[categorical_cols])
encoded_df_test = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))

df_encoded_test = pd.concat([df_test, encoded_df_test], axis=1)
df_encoded_test = df_encoded_test.drop(categorical_cols, axis=1)

df_encoded_test.head()

,total_bill,tip,size,sex_Male,smoker_Yes,day_Sat,day_Sun,day_Thur,time_Lunch
0,19.82,3.18,2,1.0,0.0,1.0,0.0,0.0,0.0
1,8.77,2.00,2,1.0,0.0,0.0,1.0,0.0,0.0
2,24.55,2.00,4,1.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
df_encoded_test.sum().sum()

74.32

# 2. Normalização

Normalize todos os atributos numérico de `df` (`tips_train.csv`) aplicando a normalização *z-score*. Com o mesmo estimador normalize os valores de `df_test` (`tips_test.csv`).

**Q3.** Qual o menor valor de `total_bill` no dataframe `df_test` (`tips_test.csv`) transformado?

<br>

**Nota**: Este exercício é independente do anterior. Carregue novamente os dados, não empregue os dados transformados.

In [ ]:
df = pd.read_csv('tips_train.csv')
df_test = pd.read_csv('tips_test.csv')
display(df.head())
display(df_test.head())

,total_bill,tip,sex,smoker,day,time,size
0,25.89,5.16,Male,Yes,Sat,Dinner,4
1,13.00,2.00,Female,Yes,Thur,Lunch,2
2,17.89,2.00,Male,Yes,Sun,Dinner,2
3,28.44,2.56,Male,Yes,Thur,Lunch,2
4,12.48,2.52,Female,No,Thur,Lunch,2


,total_bill,tip,sex,smoker,day,time,size
0,19.82,3.18,Male,No,Sat,Dinner,2
1,8.77,2.00,Male,No,Sun,Dinner,2
2,24.55,2.00,Male,No,Sun,Dinner,4


In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

numeric_cols = df.select_dtypes(include=['number']).columns

# Define
scaler =  StandardScaler()

# Fit Transform
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

display(df.head())


,total_bill,tip,sex,smoker,day,time,size
0,0.682510,1.553893,Male,Yes,Sat,Dinner,1.508205
1,-0.764871,-0.725525,Female,Yes,Thur,Lunch,-0.598910
2,-0.215787,-0.725525,Male,Yes,Sun,Dinner,-0.598910
3,0.968842,-0.321578,Male,Yes,Thur,Lunch,-0.598910
4,-0.823260,-0.350431,Female,No,Thur,Lunch,-0.598910


In [ ]:
df.describe()

,total_bill,tip,size
count,2.410000e+02,2.410000e+02,2.410000e+02
mean,-2.119098e-16,1.842694e-17,-1.980896e-16
std,1.002081e+00,1.002081e+00,1.002081e+00
min,-1.879882e+00,-1.446861e+00,-1.652468e+00
25%,-7.233246e-01,-7.255254e-01,-5.989103e-01
50%,-2.281385e-01,-6.189714e-02,-5.989103e-01
75%,4.792702e-01,4.286107e-01,4.546473e-01
max,3.480704e+00,5.045155e+00,3.615320e+00


In [ ]:
df_test[numeric_cols] = scaler.transform(df_test[numeric_cols])
display(df_test.head())

,total_bill,tip,sex,smoker,day,time,size
0,0.000927,0.125650,Male,No,Sat,Dinner,-0.598910
1,-1.239845,-0.725525,Male,No,Sun,Dinner,-0.598910
2,0.532045,-0.725525,Male,No,Sun,Dinner,1.508205


# 3. Regressão Logística

Faça um modelo de **regressão Logística** para prever os valores de `tip_label` **com base nos demais atributos que você considere preditores** (tenha atenção aqui, nem todos os atributos são preditores). Para a separação de dados de treinamento e teste, empregue:
```
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
```
e empregue o **número máximo de iterações do estimador logístico em 1000**.

**Q4.** Qual a acuracidade do modelo obtida?

**Q5.** Qual a predição obtida para os 3 novos casos?

<br>

**nota**: O encode dos dados já foi aqui realizado antes e não é necessário normalizar os dados para este exercício (a normalização não altera significativamente os resultados aqui).

In [ ]:
df = pd.read_csv('tips_train_2.csv')
df_new = pd.read_csv('tips_test_2.csv')
display(df.head())
display(df_new.head())

,credit_card,total_bill,Sunday,sex,time,size,tip_label
0,6834,25.89,0,0,1,4,> 0.12
1,5759,13.00,0,1,0,2,> 0.12
2,9697,17.89,1,0,1,2,<=0.12
3,2332,28.44,0,0,0,2,<=0.12
4,7108,12.48,0,1,0,2,> 0.12


,credit_card,total_bill,Sunday,sex,time,size,tip_label
0,7125,19.82,0,0,1,2,> 0.12
1,7345,8.77,1,0,1,2,> 0.12
2,8575,24.55,1,0,1,4,<=0.12


In [ ]:
from sklearn.linear_model import LogisticRegression

# Separar as features (X) e o target (y)
X = df.drop(['tip_label','credit_card'], axis=1)
y = df['tip_label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criar o modelo de regressão logística
model = LogisticRegression(max_iter=1000)

# Treinar o modelo
model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Calcular a acurácia do modelo
accuracy = model.score(X_test, y_test)
print("Acurácia do modelo:", accuracy)



Acurácia do modelo: 0.8571428571428571


In [ ]:
# Fazer previsões no conjunto de teste
y_pred = model.predict(df_new.drop(['tip_label','credit_card'], axis=1))

df_new['predicted_tip_label'] = y_pred
df_new

,credit_card,total_bill,Sunday,sex,time,size,tip_label,predicted_tip_label
0,7125,19.82,0,0,1,2,> 0.12,> 0.12
1,7345,8.77,1,0,1,2,> 0.12,> 0.12
2,8575,24.55,1,0,1,4,<=0.12,> 0.12


# 4. K-Vizinhos mais Próximos

Faça um modelo de **K-Vizinhos mais Próximos** para prever os valores de `tip_label` **com base nos demais atributos que você considere preditores** (tenha atenção aqui, nem todos os atributos são preditores). Para a separação de dados de treinamento e teste, empregue:
```
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
```
e empregue **k=9**.

**Q6.** Qual a acuracidade do modelo obtida?

**Q7.** Qual a predição obtida para os 3 novos casos?

**Q8.** Verifique qual classe apresenta mais **falsos negativos**. Qual é esse número de falsos negativos?

<br>

**nota**: O encode dos dados já foi aqui realizado antes e não é necessário normalizar os dados para este exercício (a normalização não altera significativamente os resultados aqui).

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Separar as features (X) e o target (y)
X = df.drop(['tip_label','credit_card'], axis=1)
y = df['tip_label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criar o modelo de regressão logística
model = KNeighborsClassifier(9)

# Treinar o modelo
model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Calcular a acurácia do modelo
accuracy = model.score(X_test, y_test)
print("Acurácia do modelo:", accuracy)



Acurácia do modelo: 0.8163265306122449


In [ ]:
# Fazer previsões no conjunto de teste
y_pred = model.predict(df_new.drop(['tip_label','credit_card','predicted_tip_label'], axis=1))

df_new['predicted_tip_label'] = y_pred
df_new

,credit_card,total_bill,Sunday,sex,time,size,tip_label,predicted_tip_label
0,7125,19.82,0,0,1,2,> 0.12,> 0.12
1,7345,8.77,1,0,1,2,> 0.12,> 0.12
2,8575,24.55,1,0,1,4,<=0.12,> 0.12


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Calcular a matriz de confusão
y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)

# Imprimir a matriz de confusão
print("Matriz de Confusão:\n", cm)

# Gerar o classification report
cr = classification_report(y_test, y_pred)

# Imprimir o classification report
print("\nClassification Report:\n", cr)


Matriz de Confusão:
 [[ 1  6]
 [ 3 39]]

Classification Report:
               precision    recall  f1-score   support

      <=0.12       0.25      0.14      0.18         7
      > 0.12       0.87      0.93      0.90        42

    accuracy                           0.82        49
   macro avg       0.56      0.54      0.54        49
weighted avg       0.78      0.82      0.79        49



In [ ]:
0.5 + 8*1.5

12.5